In [25]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
import os
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import func,case


In [2]:
app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///../data/new_olympics.db'
db = SQLAlchemy(app)



C:\Users\norfe\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
sqlite3.connect('../data/olympics_raw.sqlite')
conn = sqlite3.connect('../data/new_olympics.db')

In [4]:
pd.read_sql('SELECT * FROM olympics_raw LIMIT 1', conn)

,id,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal,country_id
0,1,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold,1


In [5]:
 # Create engine using the `demographics.sqlite` database file
Base = automap_base()    
engine = create_engine("sqlite:///../data/new_olympics.db")
Base.prepare(engine, reflect=True)

In [6]:
Base.classes.keys()

['country_ref', 'olympics_raw']

In [7]:
Base.metadata

MetaData(bind=None)

In [8]:
class olympic_raw(db.Model):
    __tablename__ = 'olympics_raw'
    __table_args__ = {'extend_existing': True} 
    id = db.Column(db.Integer, primary_key=True)
    City = db.Column(db.String(64))
    Edition =  db.Column(db.String(64))
    Event= db.Column(db.String(64))
    Event_Gender= db.Column(db.String(64))
    Medal = db.Column(db.String(64))
    country_id =db.Column(db.Integer)
    NOC = db.Column(db.String(64))
    Gender = db.Column(db.String(64))
    Discipline = db.Column(db.String(64))
    Athlete = db.Column(db.String(64))

In [9]:
class raw(db.Model):
    __tablename__ = 'olympics_raw'
    __table_args__ = {'extend_existing': True} 
    id = db.Column(db.Integer, primary_key=True)
    City = db.Column(db.String(64))
    Edition =  db.Column(db.String(64))
    Event= db.Column(db.String(64))
    Event_Gender= db.Column(db.String(64))
    Medal = db.Column(db.String(64))
    country_id =db.Column(db.Integer, db.ForeignKey('country_ref.id'))
    NOC = db.Column(db.String(64))
    Gender = db.Column(db.String(64))
    Discipline = db.Column(db.String(64))
    Athlete = db.Column(db.String(64))
    Sport = db.Column(db.String(64))
    def __repr__(self):
        return '<raw %r>' % (self.id)

In [10]:
class country_ref(db.Model):
    __tablename__ = 'country_ref' 
    __table_args__ = {'extend_existing': True} 
    id = db.Column(db.Integer, primary_key=True)
    country_name = db.Column(db.String(64))
    code =db.Column(db.String(64))
    flag_source = db.Column(db.String(64))
    flag_image = db.Column(db.String(64))
    def __repr__(self):
        return '<country_ref %r>' % (self.id)


In [11]:
countries = Base.classes.country_ref


In [12]:
cr = db.session.query(country_ref.id, country_ref.code, country_ref.country_name,country_ref.flag_image).all()


In [13]:
count = db.session.query(raw.NOC,raw.Edition, raw.Sport, func.count(raw.id)).group_by(raw.Edition,raw.Sport).filter(raw.NOC == 'USA').all()
count

[('USA', '1896', 'Athletics', 17),
 ('USA', '1896', 'Shooting', 3),
 ('USA', '1900', 'Athletics', 39),
 ('USA', '1900', 'Cycling', 1),
 ('USA', '1900', 'Golf', 4),
 ('USA', '1900', 'Rowing', 9),
 ('USA', '1900', 'Sailing', 1),
 ('USA', '1900', 'Tennis', 1),
 ('USA', '1904', 'Aquatics', 28),
 ('USA', '1904', 'Archery', 30),
 ('USA', '1904', 'Athletics', 72),
 ('USA', '1904', 'Boxing', 19),
 ('USA', '1904', 'Cycling', 21),
 ('USA', '1904', 'Fencing', 6),
 ('USA', '1904', 'Football', 46),
 ('USA', '1904', 'Golf', 23),
 ('USA', '1904', 'Gymnastics', 44),
 ('USA', '1904', 'Lacrosse', 13),
 ('USA', '1904', 'Roque', 3),
 ('USA', '1904', 'Rowing', 36),
 ('USA', '1904', 'Tennis', 12),
 ('USA', '1904', 'Tug of War', 15),
 ('USA', '1904', 'Weightlifting', 5),
 ('USA', '1904', 'Wrestling', 21),
 ('USA', '1908', 'Aquatics', 6),
 ('USA', '1908', 'Archery', 1),
 ('USA', '1908', 'Athletics', 39),
 ('USA', '1908', 'Jeu de paume', 1),
 ('USA', '1908', 'Shooting', 14),
 ('USA', '1908', 'Wrestling', 2),
 

In [14]:
subq = (db.session.query(raw.country_id,raw.Medal,func.count(raw.id)).group_by(raw.country_id,raw.Medal)).filter(raw.Medal=='Gold').subquery()

In [15]:
db.session.query(subq).all()

[(1, 'Gold', 400),
 (2, 'Gold', 21),
 (3, 'Gold', 34),
 (4, 'Gold', 2088),
 (5, 'Gold', 407),
 (6, 'Gold', 498),
 (7, 'Gold', 378),
 (8, 'Gold', 293),
 (9, 'Gold', 147),
 (10, 'Gold', 73),
 (11, 'Gold', 23),
 (12, 'Gold', 212),
 (13, 'Gold', 91),
 (14, 'Gold', 128),
 (15, 'Gold', 154),
 (17, 'Gold', 347),
 (18, 'Gold', 194),
 (19, 'Gold', 92),
 (20, 'Gold', 460),
 (21, 'Gold', 160),
 (22, 'Gold', 20),
 (23, 'Gold', 24),
 (24, 'Gold', 124),
 (25, 'Gold', 1),
 (26, 'Gold', 9),
 (27, 'Gold', 80),
 (28, 'Gold', 76),
 (29, 'Gold', 59),
 (30, 'Gold', 206),
 (31, 'Gold', 1),
 (32, 'Gold', 68),
 (33, 'Gold', 103),
 (34, 'Gold', 4),
 (35, 'Gold', 44),
 (36, 'Gold', 143),
 (37, 'Gold', 155),
 (39, 'Gold', 7),
 (41, 'Gold', 8),
 (42, 'Gold', 3),
 (43, 'Gold', 2),
 (44, 'Gold', 14),
 (45, 'Gold', 37),
 (46, 'Gold', 1),
 (47, 'Gold', 23),
 (49, 'Gold', 140),
 (51, 'Gold', 1),
 (52, 'Gold', 11),
 (53, 'Gold', 1),
 (54, 'Gold', 838),
 (55, 'Gold', 1),
 (56, 'Gold', 53),
 (58, 'Gold', 68),
 (60, 'Gold

In [16]:
db.session.query(country_ref.country_name, country_ref.code, subq).join(subq).all()

[('Hungary', 'HUN', 1, 'Gold', 400),
 ('Austria', 'AUT', 2, 'Gold', 21),
 ('Greece', 'GRE', 3, 'Gold', 34),
 ('United States', 'USA', 4, 'Gold', 2088),
 ('Germany', 'GER', 5, 'Gold', 407),
 ('United Kingdom', 'GBR', 6, 'Gold', 498),
 ('France', 'FRA', 7, 'Gold', 378),
 ('Australia', 'AUS', 8, 'Gold', 293),
 ('Denmark', 'DEN', 9, 'Gold', 147),
 ('Switzerland', 'SUI', 10, 'Gold', 73),
 ('Mixed Teams', 'ZZX', 11, 'Gold', 23),
 ('Netherlands', 'NED', 12, 'Gold', 212),
 ('Belgium', 'BEL', 13, 'Gold', 91),
 ('India', 'IND', 14, 'Gold', 128),
 ('Canada', 'CAN', 15, 'Gold', 154),
 ('Sweden', 'SWE', 17, 'Gold', 347),
 ('Norway', 'NOR', 18, 'Gold', 194),
 ('Spain', 'ESP', 19, 'Gold', 92),
 ('Italy', 'ITA', 20, 'Gold', 460),
 ('Cuba', 'CUB', 21, 'Gold', 160),
 ('Australasia', 'ANZ', 22, 'Gold', 20),
 ('South Africa', 'RSA', 23, 'Gold', 24),
 ('Finland', 'FIN', 24, 'Gold', 124),
 ('Russian Empire', 'RU1', 25, 'Gold', 1),
 ('Estonia', 'EST', 26, 'Gold', 9),
 ('Czechoslovakia', 'TCH', 27, 'Gold', 80

In [17]:
db.session.query(country_ref,count).filter(country_ref.id == count.country_id).all()

InvalidRequestError: SQL expression, column, or mapped entity expected - got '[('USA', '1896', 'Athletics', 17), ('USA', '1896', 'Shooting', 3), ('USA', '1900', 'Athletics', 39), ('USA', '1900', 'Cycling', 1), ('USA', '1900', 'Golf', 4), ('USA', '1900', 'Rowing', 9), ('USA', '1900', 'Sailing', 1), ('USA', '1900', 'Tennis', 1), ('USA', '1904', 'Aquatics', 28), ('USA', '1904', 'Archery', 30), ('USA', '1904', 'Athletics', 72), ('USA', '1904', 'Boxing', 19), ('USA', '1904', 'Cycling', 21), ('USA', '1904', 'Fencing', 6), ('USA', '1904', 'Football', 46), ('USA', '1904', 'Golf', 23), ('USA', '1904', 'Gymnastics', 44), ('USA', '1904', 'Lacrosse', 13), ('USA', '1904', 'Roque', 3), ('USA', '1904', 'Rowing', 36), ('USA', '1904', 'Tennis', 12), ('USA', '1904', 'Tug of War', 15), ('USA', '1904', 'Weightlifting', 5), ('USA', '1904', 'Wrestling', 21), ('USA', '1908', 'Aquatics', 6), ('USA', '1908', 'Archery', 1), ('USA', '1908', 'Athletics', 39), ('USA', '1908', 'Jeu de paume', 1), ('USA', '1908', 'S

In [ ]:
results=db.session.query(country_ref.country_name, country_ref.code, subq).join(subq).all()
results

In [ ]:
  country_dic ={}

In [ ]:
for result in results:
    if result[2] in country_dic:
        country_dic[result[2]]['medals'][result[3]] = result[4]
    else:
        country ={}

        country['name']  = result[0]
        country['code'] = result[1]
        country['medals'] = {}
        country['medals'][result[3]] = result[4]
        country_dic[result[2]] = country

In [26]:
goldCase = case([(raw.Medal == 'Gold', 1)], else_=0)

In [27]:
db.session.query(raw.Edition, raw.Gender, raw.Sport,func.count(raw.id).label('totMed'),func.sum(goldCase)).group_by(raw.Edition,raw.Gender,raw.Sport).filter(country_ref.code=='FRA').all()


[('1896', 'Men', 'Aquatics', 11, 4),
 ('1896', 'Men', 'Athletics', 37, 12),
 ('1896', 'Men', 'Cycling', 16, 6),
 ('1896', 'Men', 'Fencing', 8, 3),
 ('1896', 'Men', 'Gymnastics', 45, 28),
 ('1896', 'Men', 'Shooting', 15, 5),
 ('1896', 'Men', 'Tennis', 10, 3),
 ('1896', 'Men', 'Weightlifting', 6, 2),
 ('1896', 'Men', 'Wrestling', 3, 1),
 ('1900', 'Men', 'Aquatics', 61, 18),
 ('1900', 'Men', 'Archery', 18, 6),
 ('1900', 'Men', 'Athletics', 76, 27),
 ('1900', 'Men', 'Basque Pelota', 4, 2),
 ('1900', 'Men', 'Cricket', 24, 12),
 ('1900', 'Men', 'Croquet', 8, 4),
 ('1900', 'Men', 'Cycling', 6, 2),
 ('1900', 'Men', 'Equestrian', 9, 4),
 ('1900', 'Men', 'Fencing', 21, 7),
 ('1900', 'Men', 'Football', 35, 11),
 ('1900', 'Men', 'Golf', 3, 1),
 ('1900', 'Men', 'Gymnastics', 3, 1),
 ('1900', 'Men', 'Polo', 12, 4),
 ('1900', 'Men', 'Rowing', 68, 23),
 ('1900', 'Men', 'Rugby', 47, 17),
 ('1900', 'Men', 'Sailing', 26, 9),
 ('1900', 'Men', 'Shooting', 52, 17),
 ('1900', 'Men', 'Tennis', 16, 4),
 ('1900

In [ ]:
data =db.session.query(country_ref.code, country_ref.country_name, country_ref.id,country_ref.flag_image).filter(country_ref.code == 'FRA').all()

In [ ]:
country_data = db.session.query(country_ref.id, country_ref.code, country_ref.country_name,country_ref.flag_image).filter(country_ref.code== 'FRA').all()
  

In [ ]:
 word_cloud = db.session.query(raw.Sport,func.count(raw.id).label('nMed')).group_by(raw.Sport).filter(raw.country_id== country_data[0].id)

In [ ]:
words = []
for sport in word_cloud:
    sp ={}
    sp['word'] = sport.Sport
    

In [ ]:
help(word_cloud)

In [ ]:
word_cloud[0].nMed